**Importing Libraries and Data**

In [ ]:
#Importing libraries
import pandas as pd
import statsmodels.api as sm
from sklearn.metrics import root_mean_squared_error
from statsmodels.tsa.stattools import acf
import matplotlib.pyplot as plt
import prophet
import numpy as np
import warnings
import gc
import os
import sys

#Ignoring warnings
warnings.filterwarnings("ignore")

#Reading in data and extracting unique account values
BoostedTransactions = pd.read_csv("Boosted Transaction Dataset.csv")
AccountIDs = BoostedTransactions["AccountId"].values
UniqueAccounts = set(AccountIDs)

#Converting transaction dates into months
BoostedTransactions["TransactionDate"] = pd.to_datetime(BoostedTransactions["TransactionDate"])
BoostedTransactions["TransactionDate"] = BoostedTransactions["TransactionDate"].dt.to_period("M").dt.to_timestamp()

#Grouping transactions by account and month and summating the net transaction amount for each grouping
GroupedSumOfTransactionsByAccountAndMonth = BoostedTransactions.groupby(by=["AccountId", "TransactionDate"]).agg(["sum"])["Amount"]
print(GroupedSumOfTransactionsByAccountAndMonth)

#Creating a list to store all account time series in
DataFrames = []

#Imputing 0 values into gaps in each account's time series
for i in range(0, len(UniqueAccounts)):
  ResetDataFrame = GroupedSumOfTransactionsByAccountAndMonth.reset_index()
  TimeSeries = ResetDataFrame[ResetDataFrame["AccountId"] == list(UniqueAccounts)[i]]
  TimeSeries = TimeSeries.set_index("TransactionDate")

  TimeSeries = TimeSeries.resample("MS").sum()
  for j in range(0, len(TimeSeries)):
    if TimeSeries.iloc[j]["AccountId"] == 0:
      TimeSeries.at[TimeSeries.index[j], "AccountId"] = list(UniqueAccounts)[i]

  DataFrames.append(TimeSeries)

#Combining all accounts' time series into a new grouped dataset
NewGroupedDataset = pd.concat(DataFrames)
NewGroupedDataset = NewGroupedDataset.reset_index()
NewGroupedDataset = NewGroupedDataset.set_index(["AccountId", "TransactionDate"])
print(NewGroupedDataset)

#Calculating the cumulative transaction sum over the time period for each account's time series
GroupedCumulativeSumOfTransactionsByAccountAndMonth = NewGroupedDataset.groupby(level=0).cumsum().reset_index()
print(GroupedCumulativeSumOfTransactionsByAccountAndMonth)

                                                          sum
AccountId                            TransactionDate         
0003a5ae-0c77-4372-b44d-882ef9874a28 2019-05-01      -5541.31
                                     2019-06-01      -3601.46
                                     2019-07-01       3151.74
                                     2019-08-01      -4801.13
                                     2019-09-01       3890.05
...                                                       ...
fff7f00c-c869-4310-b705-4503538f5ecf 2020-03-01       2974.84
                                     2020-04-01        382.93
                                     2020-05-01        166.15
                                     2020-06-01       -550.89
                                     2020-07-01      -2721.18

[10995 rows x 1 columns]
                                                          sum
AccountId                            TransactionDate         
7a6a6cde-6586-4f4d-97e6-ff371d7bd426 2019-05

**Key Prophet Loop**

In [ ]:
#Creating models and forecasting for all accounts
for i in UniqueAccounts:
    print(i)
    try:
        #Selecting only the selected account's time series
        ForecastingTimeSeries = GroupedCumulativeSumOfTransactionsByAccountAndMonth[GroupedCumulativeSumOfTransactionsByAccountAndMonth["AccountId"] == i][["TransactionDate", "sum"]]

        #Calculating the cutoff between training and testing sets
        TrainEnd = int(0.7 * len(ForecastingTimeSeries))

        #Splitting the data into training and testing sets and resetting column names in each
        TrainData = ForecastingTimeSeries[:TrainEnd]
        TestData = ForecastingTimeSeries[TrainEnd:]

        TrainData.columns = ["ds", "y"]
        TestData.columns = ["ds", "y"]

        #Declaring and training the model
        Model = prophet.Prophet()
        Model.fit(TrainData)

        #Forecasting the test data
        Forecast = Model.predict(TestData)

        #Calculating error rates and appending them into the errors CSV
        try:
          #print(IdealNumberOfStreams)
          RMSE = root_mean_squared_error(Forecast["yhat"], TestData["y"])
          SI = abs(RMSE/abs(TestData["y"]).mean())
          AccountError = pd.DataFrame({"AccountID" : [i],
                                        "RMSE" : [RMSE],
                                        "SI" : [SI]})
          AccountError.to_csv("Prophet Errors (Cumulative).csv", mode='a', header=not os.path.exists("Prophet Errors (Cumulative).csv"), index=False)
        #Where a ValueError occurs, the loop is broken
        except ValueError:
          print("ValueError encountered")
          break

        #Forecasting the next 12 months
        Future = Model.make_future_dataframe(periods=12, freq="MS")
        FutureForecast = Model.predict(Future)

        FutureForecastTail = FutureForecast.tail(12)
        FutureForecastValues = FutureForecastTail["yhat"].values

        #Calculating starting amount, ending amount and net change and appending them into the forecasts CSV
        NetChange = FutureForecastValues[-1] - FutureForecastValues[0]
        StartingAmount = FutureForecastValues[0]
        EndingAmount = FutureForecastValues[-1]

        ForecastData = pd.DataFrame({"AccountID" : [i],
                                     "Net Change over Forecast Period" : [NetChange],
                                     "Starting Amount" : [StartingAmount],
                                     "Ending Amount" : [EndingAmount]})

        ForecastData.to_csv("Prophet Forecasts (Cumulative).csv", mode='a', header=not os.path.exists("Prophet Forecasts (Cumulative).csv"), index=False)

        print("Processed account number",i)
    #Where an error occurs, the user is informed about it
    except Exception as e:
        print(f"Error encountered processing account number {i}.")
        ErrorType, ErrorObject, ErrorTraceback = sys.exc_info()

        ErrorFilename = os.path.split(
            ErrorTraceback.tb_frame.f_code.co_filename
        )[1]

        ErrorMessage = str(e)

        ErrorLineNumber = ErrorTraceback.tb_lineno

        print(f'Exception Type: {ErrorType}')

        print(f'Exception Filename: {ErrorFilename}')

        print(f'Exception Line Number: {ErrorLineNumber}')

        print(f'Exception Message: {ErrorMessage}')
        break
    #Deleting model data to conserve RAM
    finally:
        del Model, Forecast, TrainData, TestData, ForecastingTimeSeries
        gc.collect()


7a6a6cde-6586-4f4d-97e6-ff371d7bd426


INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:n_changepoints greater than number of observations. Using 6.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpf1qjq47e/7m7hrhco.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpf1qjq47e/37m6thzf.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.11/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=96355', 'data', 'file=/tmp/tmpf1qjq47e/7m7hrhco.json', 'init=/tmp/tmpf1qjq47e/37m6thzf.json', 'output', 'file=/tmp/tmpf1qjq47e/prophet_modelbtwgy2ep/prophet_model-20250807110556.csv', 'method=optimize', 'algorithm=newton', 'iter=10000']
11:05:56 - cmdstanpy - INFO - Chain [1] start processing
IN

Processed account number 7a6a6cde-6586-4f4d-97e6-ff371d7bd426
b58a892c-da69-4241-8d26-9ae4cc1c4816
Processed account number b58a892c-da69-4241-8d26-9ae4cc1c4816


INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:n_changepoints greater than number of observations. Using 7.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpf1qjq47e/0ve4zes5.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpf1qjq47e/kgds9sow.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.11/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=87505', 'data', 'file=/tmp/tmpf1qjq47e/0ve4zes5.json', 'init=/tmp/tmpf1qjq47e/kgds9sow.json', 'output', 'file=/tmp/tmpf1qjq47e/prophet_model27cnvlbc/prophet_model-20250807110557.csv', 'method=optimize', 'algorithm=newton', 'iter=10000']
11:05:57 - cmdstanpy - INFO - Chain [1] start processing
IN

64619d89-79d1-4320-ab20-ff5102ac56f1
Processed account number 64619d89-79d1-4320-ab20-ff5102ac56f1


...

Processed account number 0a2c2b45-42bf-4a2e-8118-e7279e5589de
c3beeaa8-87ce-455d-9482-e01679b9d1eb


11:10:47 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:n_changepoints greater than number of observations. Using 7.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpf1qjq47e/bk4qeob7.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpf1qjq47e/qsw82fpx.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.11/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=48369', 'data', 'file=/tmp/tmpf1qjq47e/bk4qeob7.json', 'init=/tmp/tmpf1qjq47e/qsw82fpx.json', 'output', 'file=/tmp/tmpf1qjq47e/prophet_modeli2bpqej1/prophet_model-20250807111047.csv', 'method=optimiz

Processed account number c3beeaa8-87ce-455d-9482-e01679b9d1eb
92937092-480e-4b7a-87ad-01898b28c9e0


11:10:47 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


Processed account number 92937092-480e-4b7a-87ad-01898b28c9e0


**Calculating Error Statistics**

In [ ]:
#Calculating averaged error statistics
Errors = pd.read_csv("Prophet Errors (Cumulative).csv")
MeanRMSE = Errors["RMSE"].mean()
print("Mean RMSE:",MeanRMSE)
MedianRMSE = Errors["RMSE"].median()
print("Median RMSE:",MedianRMSE)
MeanSI = Errors["SI"].mean()
print("Mean SI:",MeanSI)
MedianSI = Errors["SI"].median()
print("Median SI:",MedianSI)

Mean RMSE: 106535.88003839368
Median RMSE: 8636.12527831311
Mean SI: 0.6027111972383336
Median SI: 0.35354600513191825
